#Question 2
Collect some data (e.g., from a friend or your teammate) on the recognition memory task from the
slides (or construct your own task) and fit the following two models using Stan:
• The One-High-Threshold Model (1HT)
• The Two-High-Threshold Model (2HT)
The models are depcited on Slide 12 (MPT models). As usual, inspect the convergence of the MCMC
samplers and report the estimation results. Do the two models suggest different estimates for the
two key parameters? Describe and interpret the results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
##import seaborn as sns
import pandas as pd
import pystan as stan

In [3]:
df = pd.read_csv('MRT_Data.csv', delimiter=';',dtype={'old_new': int, 'choice': int, 'accuracy': int})
##print for debug
print(df)

    old_new  remember  accuracy
0         0         0         1
1         0         0         1
2         1         1         1
3         1         1         1
4         0         0         1
5         0         0         1
6         1         1         1
7         0         0         1
8         1         1         1
9         1         1         1
10        0         0         1
11        0         0         1
12        0         0         1
13        1         1         1
14        1         0         1
15        0         0         1
16        1         1         1
17        0         1         0
18        1         1         1
19        1         1         1


In [29]:
OneHT_program_code = """
data {
    int<lower=0> N; // Number of data points
    int<lower=0, upper=1> is_old[N]; // Indicator for old (1) or new (0) word
    int<lower=0, upper=1> is_confident[N]; // Indicator for confident (1) or not (0)
    int<lower=0, upper=1> is_correct[N]; // Indicator for correct (1) or wrong (0) response
}
parameters {
    real mu;        // Mean
    real<lower=0> sigma; // Standard deviation
    real<lower=0, upper=1> pho; // Probability of remembering a studied item on the old list
    real<lower=0, upper=1> gamma; // Probability of guessing old when no memory
    
}
model {
    //prior distributions
    pho ~ beta(1,1);
    gamma ~ beta(1,1);
    for (i in 1:N) {
        if (is_old[i] == 1&&is_correct[i]==1) {
            is_confident[i] ~ bernoulli(pho);
            if(is_confident[i]==0){1~bernoulli(gamma);} 
        }
        else{
            if(is_correct[i]==1){0~bernoulli(gamma);}
            else{1~bernoulli(gamma);}
        }
    }
}
"""

In [41]:
TwoHT_program_code = """
data {
    int<lower=0> N; // Number of data points
    int<lower=0, upper=1> is_old[N]; // Indicator for old (1) or new (0) word
    int<lower=0, upper=1> is_confident[N]; // Indicator for confident (1) or not (0)
    int<lower=0, upper=1> is_correct[N]; // Indicator for correct (1) or wrong (0) response
}
parameters {
    real mu;        // Mean
    real<lower=0> sigma; // Standard deviation
    real<lower=0, upper=1> pho; // Probability of remembering a studied item
    real<lower=0, upper=1> gamma; // Probability of guessing old when no memory
}
model {
//prior distributions
    pho ~ beta(1,1);
    gamma ~ beta(1,1);
    for (i in 1:N) {
        if(is_correct[i]==1){
            is_confident[i] ~ bernoulli(pho);
            if(is_confident[i]==0){
                if(is_old[i]==1){
                    1~bernoulli(gamma);
                }
                else{0~bernoulli(gamma);}
            }
        }
        else{
            0~bernoulli(pho);
            if(is_old[i]==1){0~bernoulli(gamma);}
            else{1~bernoulli(gamma);}
        }
    }
}
"""

In [42]:
OneHT_model = stan.StanModel(model_code=OneHT_program_code)
TwoHT_model = stan.StanModel(model_code=TwoHT_program_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1e7208daf67add1d04f08e2ac216baa9 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f28819db432812417a658dcf31f1b9b3 NOW.


In [43]:
##data preparation
data={
    'N':len(df),
    'is_old':df['old_new'],
    'is_confident':df['remember'],
    'is_correct': df['accuracy']
}

In [44]:
OneHT_fit = OneHT_model.sampling(data=data, chains=4, iter=4000,control={'adapt_delta': 0.99})
TwoHT_fit = TwoHT_model.sampling(data=data, chains=4, iter=4000,control={'adapt_delta': 0.99})

#convergence inspection and estimations
print(OneHT_fit)
print(TwoHT_fit)


Inference for Stan model: anon_model_1e7208daf67add1d04f08e2ac216baa9.
4 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=8000.

        mean se_mean     sd    2.5%     25%     50%     75%   97.5%  n_eff   Rhat
mu     1.3e4   6.7e4  1.2e5  -2.6e5   -3218  8090.5   3.6e4   2.5e5      3   3.27
sigma    inf     nan    inf 5.1e306 4.6e307 9.2e307 1.4e308 1.8e308    nan    nan
pho     0.84  4.0e-3    0.1    0.59    0.78    0.86    0.91    0.98    644    1.0
gamma   0.23  4.1e-3   0.11    0.05    0.15    0.22     0.3    0.48    733    1.0
lp__  695.33    0.06    1.4  691.84   694.6  695.63  696.38  697.13    514    1.0

Samples were drawn using NUTS at Fri Apr 12 00:01:30 2024.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
Inference for Stan model: anon_model_f28819db432812417a658dcf31f1b9b3.
4 chains, each with iter=

c:\Users\ken12\anaconda3\envs\newstan_env\lib\site-packages\numpy\core\_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\ken12\anaconda3\envs\newstan_env\lib\site-packages\numpy\core\_methods.py:212: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


#Discussion
While not the prettiest results, I never the less achieved reasonable convergence on my models, as shown by the 1.0 rhat results above. The results are unsurprising. For the One High Threshold Model, the data I collected suggested a 98% probability for a participant confidently remembering a word for the old list, and a 48% chance that they would guess that word was from the old list. With the Two-High model, gamma remained unchanged, which makes sense given that the parameter is measuring the liklihood of the same event. Meanwhile pho now represents the probability of a correct confident response, and has fallen to 66%. I imagine my data is to blame, as my subject was unconfident remembering every word from the new list (Results like this may also indicate a lack of clarity in my instruction). I would be interested to see what these results would be like if I clarified my instruction and/or used different subjects, so I will be repeating this experiment, with some additional changes involving a distraction task, for my cognitive psych take home assignment.